In [3]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.metrics import mean_squared_error, mean_absolute_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import random
import glob

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Chemin vers les fichiers CSV
chemin = '/content/players/*.csv'

# Liste des noms de fichiers
fichiers = glob.glob(chemin)

# Initialisation d'une liste pour stocker les DataFrames de chaque fichier
dfs = []
# Liste des encodages à essayer
encodings_to_try = [ 'utf-8','ISO-8859-1']

# Charger et concaténer les fichiers
for fichier in fichiers:
    df = None
    # Essayer différents encodages jusqu'à ce que la lecture réussisse
    for encoding in encodings_to_try:
        try:
            df = pd.read_csv(fichier, encoding=encoding)
            print(f"Lecture réussie avec l'encodage {encoding}.")
            break  # Sortir de la boucle si la lecture réussit avec un encodage
        except UnicodeDecodeError:
            print(f"Échec de la lecture avec l'encodage {encoding}. Essayer un autre encodage.")

    if df is not None:
        # Ajouter une colonne 'source' pour identifier la provenance de chaque fichier
        nom_fichier = fichier.split('/')[-1]
        saison = nom_fichier.split('.')[0].split('-')
        df['source'] = f"{saison[0]}-{saison[1]}"
        # Concaténer 'first_name' et 'second_name' en une seule colonne 'name'
        df['name'] = df['first_name'] + ' ' + df['second_name']
        # Supprimer les colonnes 'first_name' et 'second_name'
        df = df.drop(columns=['first_name', 'second_name'])
        # Ajouter le DataFrame à la liste
        dfs.append(df)
    else:
        print("Échec de la lecture du fichier:", fichier)


# Concaténer tous les DataFrames en un seul
data = pd.concat(dfs, ignore_index=True)



# Maintenant vous pouvez faire du traitement de données sur le DataFrame 'data'
# Par exemple, afficher les premières lignes
print(data.info())


In [ ]:
# Ingénierie des caractéristiques
selected_features = ['goals_scored','assists','goals_conceded','creativity','influence','threat','bonus','clean_sheets','red_cards','yellow_cards','now_cost']
X = data[selected_features]
y = data['total_points']

In [ ]:
# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Sélectionner uniquement les colonnes numériques
numeric_columns = X_train.select_dtypes(include=['number']).columns

In [ ]:
# Vérification des valeurs manquantes dans les données d'entraînement
print("Valeurs manquantes dans y_train:", np.isnan(y_train).sum())

# Vérification des valeurs manquantes dans les données de test
print("Valeurs manquantes dans y_test:", np.isnan(y_test).sum())


In [ ]:
# Définir les hyperparamètres à optimiser
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_iter': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'min_samples_leaf': [1, 2, 4],
    'l2_regularization': [0.0, 0.1, 0.2]
}


In [ ]:
# Create the HistGradientBoostingRegressor model
model = HistGradientBoostingRegressor()

In [ ]:
# Train the model
model.fit(X_train, y_train)


In [ ]:
# Predict on the test set
y_pred = model.predict(X_test)

In [ ]:
# Créer l'objet GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)


Evaluation des performances du modèle

In [ ]:
# Effectuer la validation croisée
scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')


In [ ]:
# Exécuter la recherche par grille sur les données
grid_search.fit(X, y)

In [ ]:
# Calcul des métriques d'évaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
# Calculer la moyenne des scores de MSE
avg_mse = -scores.mean()

In [ ]:
# Afficher les meilleurs paramètres et la meilleure MSE
print("Meilleurs paramètres:", grid_search.best_params_)
print("Meilleure MSE:", -grid_search.best_score_)

In [ ]:
print("Mean Squared Error:", mse)
print("R-squared:", r2)
print("Mean Absolute Error:", mae)

# Afficher le score moyen de MSE
print("Moyenne des scores de MSE:", avg_mse)

In [ ]:
# Tracer les valeurs réelles par rapport aux valeurs prédites
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, color='blue', label='Valeurs prédites vs réelles')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--', label='Ligne de référence')
plt.xlabel('Valeurs réelles')
plt.ylabel('Valeurs prédites')
plt.title('Comparaison des valeurs prédites aux valeurs réelles')
plt.legend()
plt.show()

In [ ]:
# Créer un nouveau modèle avec les paramètres optimisés
optimal_model = HistGradientBoostingRegressor(
    l2_regularization=0.0,
    learning_rate=0.1,
    max_depth=5,
    max_iter=300,
    min_samples_leaf=2
)


In [ ]:
# Entraîner le nouveau modèle avec les données d'entraînement
optimal_model.fit(X_train, y_train)

In [ ]:
# Prédire sur les données de test
y_pred_optimal = optimal_model.predict(X_test)

In [ ]:
# Calculer le RMSE
rmse_optimal = np.sqrt(mean_squared_error(y_test, y_pred_optimal))

# Calculer le MAPE
# Calculer le MAPE en évitant les divisions par zéro
mape_optimal = np.mean(np.abs((y_test - y_pred_optimal) / y_test.replace(0, np.nan))) * 100


In [ ]:
# Évaluer les performances du nouveau modèle sur les données de test
mse_optimal = mean_squared_error(y_test, y_pred_optimal)
r2_optimal = r2_score(y_test, y_pred_optimal)
mae_optimal = mean_absolute_error(y_test, y_pred_optimal)

In [ ]:

# Afficher les performances du nouveau modèle
print("Performances du modèle avec les paramètres optimisés :")
print("MSE:", mse_optimal)
print("R-squared:", r2_optimal)
print("MAE:", mae_optimal)
print("RMSE:", rmse_optimal)
print("MAPE:", mape_optimal)

In [ ]:
# Afficher les performances du nouveau modèle
print("Performances du modèle avec les paramètres optimisés :")
print("MSE:", mse_optimal)
print("R-squared:", r2_optimal)
print("MAE:", mae_optimal)

In [ ]:
# Tracer les valeurs réelles par rapport aux valeurs prédites
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_optimal, color='blue', label='Valeurs prédites vs réelles')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--', label='Ligne de référence')
plt.xlabel('Valeurs réelles')
plt.ylabel('Valeurs prédites')
plt.title('Comparaison des valeurs prédites aux valeurs réelles')
plt.legend()
plt.show()

In [ ]:
print(data.head())

In [ ]:
team_data = pd.read_csv('/content/gw27.csv')
team_data = team_data.rename(columns={'value': 'now_cost'})

print(team_data.head())
print(len(team_data))

In [ ]:
def predict_player_performance(model, player_name, data):
    # Filtrer les données pour obtenir les caractéristiques du joueur spécifié
    player_data = data[data['name'] == player_name]

    # Sélectionner uniquement les caractéristiques utilisées lors de l'entraînement
    selected_features = ['goals_scored','assists','goals_conceded','creativity','influence','threat','bonus','clean_sheets','red_cards','yellow_cards','now_cost']


    player_data = player_data[selected_features]

    # Effectuer la prédiction
    predicted_performance = optimal_model.predict(player_data)

    # Calculer la moyenne des performances prédites
    average_performance = np.mean(predicted_performance)

    return average_performance

In [ ]:
# Nom du joueur à prédire
player_name = 'Erling Haaland'

# Prédiction des performances du joueur
predicted_performance = predict_player_performance(optimal_model, player_name, team_data)

print("Performances prédites (total_points) pour le joueur: ",player_name, predicted_performance)


In [ ]:
# Sélectionner les informations pour le joueur spécifié
player_info = team_data[team_data['name'] == player_name]

# Vérifier si le joueur spécifié est présent dans les données
if not player_info.empty:
    # Extraire les informations nécessaires
    now_cost = player_info['now_cost'].values[0]
    total_points = player_info['total_points'].values[0]
    position = player_info['position'].values[0]  # Assurez-vous que 'element_type' contient les informations sur le poste du joueur

    # Afficher les informations
    print("Nom du joueur:", player_name)
    print("Prix actuel:", now_cost)
    print("Total de points actuel:", total_points)
    print("Poste:", position)
else:
    print("Le joueur spécifié n'a pas été trouvé dans les données.")


prediction pour les joeurus de la saison en cours

In [ ]:

# Créer une liste pour stocker les prédictions de performances pour tous les joueurs de la saison 23-24
all_predictions = []

# Itérer sur chaque joueur dans le DataFrame de la saison 23-24
for player_name in team_data['name']:
    # Prédiction des performances du joueur avec le modèle entraîné
    predicted_performance = predict_player_performance(optimal_model, player_name, team_data)

    # Ajouter le nom du joueur et ses prédictions à la liste
    all_predictions.append((player_name, predicted_performance))

# Afficher les prédictions pour tous les joueurs de la saison 23-24
for player_name, predicted_performance in all_predictions:
    print("Performances prédites (total_points) pour le joueur:", player_name, ":", predicted_performance)


In [ ]:
print(len(all_predictions))

In [ ]:
# Créer un dictionnaire à partir de la liste de toutes les prédictions
predicted_performances_dict = {player_name: predicted_performance for player_name, predicted_performance in all_predictions}


Création d'une equipe

In [ ]:
def create_team(predicted_performances, data, budget=100, max_players_per_team=3):
    team = []
    remaining_budget = budget
    positions_quota = {'GK': 2, 'DEF': 5, 'MID': 5, 'FWD': 3}
    positions_count = {'GK': 0, 'DEF': 0, 'MID': 0, 'FWD': 0}
    team_players_count = {}

    # Tri des joueurs prédits par performances décroissantes
    sorted_players = sorted(predicted_performances.items(), key=lambda x: x[1], reverse=True)

    # Sélection des joueurs pour l'équipe en respectant les contraintes
    for player_name, predicted_performance in sorted_players:
        # Vérifier si l'équipe a déjà atteint le nombre maximum de joueurs
        if len(team) >= 15:
            break

        # Vérifier si le joueur est présent dans les données
        player_data = data[data['name'] == player_name]
        if not player_data.empty:
            player_cost = player_data['now_cost'].values[0]
            player_position = player_data['position'].values[0]
            player_team = player_data['team'].values[0]

            # Vérification des contraintes de budget, de nombre maximum de joueurs par équipe et de quota par poste
            if player_cost <= remaining_budget and team_players_count.get(player_team, 0) < max_players_per_team and positions_count[player_position] < positions_quota[player_position]:
                team.append({
                    'name': player_name,
                    'position': player_position,
                    'team': player_team,
                    'cost': player_cost,
                    'predicted_performance': predicted_performance
                })
                remaining_budget -= player_cost
                positions_count[player_position] += 1
                team_players_count[player_team] = team_players_count.get(player_team, 0) + 1

    return team


In [ ]:
# Créer une équipe en utilisant les performances prédites
team = create_team(predicted_performances_dict, team_data, budget=1000, max_players_per_team=3)

In [ ]:
# Afficher l'équipe sélectionnée
print("Équipe sélectionnée:")
for player in team:
    print(f"Nom: {player['name']}, Poste: {player['position']}, Équipe: {player['team']}, Coût: {player['cost']}, Performance Prédite: {player['predicted_performance']}")


In [ ]:
# Calcul du prix total de l'équipe recommandée
total_cost = sum(player['cost'] for player in team)
total_team_points = sum(player['predicted_performance'] for player in team)
print("Le cout total de l'equipe est de:", total_cost, "k£")
print("Le nombre de point total de l'equipe est de:", total_team_points)

In [ ]:

# Affichage de la composition de l'équipe recommandée
print("Composition de l'équipe recommandée:")
for player in team:
    print("Joueur:", player['name'])
    print("Poste:", player['position'])
    print("Équipe:", player['team'])
    print("Prix:", player['cost'], 'k£')
    print("Score prédit:", player['predicted_performance'])
    print()